In [1]:
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import psycopg2 as pg2
import random
%matplotlib inline

In [2]:
user = ''
password = ''
port = ''

def get_gdf_from_postgis(query):
    con = pg2.connect(host = 'localhost', database = 'kansai_osm', user = user, password = password, port = port)
    cur = con.cursor()

    df = gpd.read_postgis(query, con, geom_col = 'way', crs = 3857)
    df = df.to_crs(epsg=4326)

    con.close()
    cur.close()
    
    return df

#### Mapping Polygons

For this example, I'm visualizing the five of the largest polygons in the *planet_osm_ploygon* table which have the largest areas. Oddly, the largest polygon present covers mainland japan as shown below and that's blows up the notebook size cause of the number of points. Hence I've omitted that polygon. Surprised it's not just the Kansai region. Anyway, polygons typically represent areas; states, counties, premises etc.

In [3]:
query = """SELECT osm_id, name, way, way_area
        FROM planet_osm_polygon
        ORDER BY way_area DESC
        LIMIT 6;"""
big_polys_df = get_gdf_from_postgis(query).iloc[1:]
big_polys_df.head()

,osm_id,name,way,way_area
1,-357113,近畿地方,"POLYGON ((134.252738 34.85189899964205, 134.25...",7.600820e+10
2,-356912,兵庫県,"POLYGON ((134.252738 34.85189899964205, 134.25...",1.875050e+10
3,-9450050,紀伊国,"POLYGON ((134.8804237 34.01222159960209, 134.8...",1.833600e+10
4,-812190,三重県,"POLYGON ((135.853184 33.85944599959755, 135.85...",1.694090e+10
5,-908959,和歌山県,"POLYGON ((134.8804237 34.01222159960209, 134.8...",1.369620e+10


In [4]:
map_center = list(reversed(list(big_polys_df.iloc[0]['way'].centroid.coords[0])))
folium_map = folium.Map(height = 550, width = 950, location = map_center, tiles = "http://a.tile.openstreetmap.fr/hot/{z}/{x}/{y}.png ", 
                        attr = "OpenStreetMaps France https://github.com/hotosm/HDM-CartoCSS", zoom_start = 7)

for i, row in big_polys_df.iterrows():
    poly_name = row['name']
    if row['name'] is None:
        poly_name = "(unknown)"
    tooltip_text = poly_name
    
    random_color = "#%06x" % random.randint(0, 0x666666)
    
    points = [(i[1], i[0]) for i in row['way'].exterior.coords]
    folium.Polygon(locations = points, tooltip = tooltip_text, popup = tooltip_text, fill=False,
                  color = random_color).add_to(folium_map)
    del points
    
folium_map